In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
users = pd.read_csv('./data/' + 'users.csv')
books = pd.read_csv('./data/' + 'books.csv')
train = pd.read_csv('./data/' + 'train_ratings.csv')
test = pd.read_csv('./data/' + 'test_ratings.csv')
sub = pd.read_csv('./data/' + 'sample_submission.csv')

In [14]:
train

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9
...,...,...,...
306790,278843,0743525493,7
306791,278851,067161746X,6
306792,278851,0884159221,7
306793,278851,0912333022,7


In [9]:
total_avg = train['rating'].mean()

In [40]:
cnt_isbn_rating = train.groupby('isbn')['rating'].count()

In [41]:
cnt_isbn_rating = cnt_isbn_rating.to_frame()

In [42]:
isbn_once_rating = cnt_isbn_rating[cnt_isbn_rating['rating'] <= 1].index

In [43]:
isbn_once_rating

Index(['0000913154', '000104687X', '0001047213', '000104799X', '0001053736',
       '0001053744', '0001055607', '0001061127', '0001360469', '0001374869',
       ...
       'B0000VZEJQ', 'B0000X8HIE', 'B00011SOXI', 'B00013AX9E', 'B00014WRH4',
       'B00019U4II', 'B0001FZGBC', 'B0001FZGPI', 'B0001FZGRQ', 'B000234N3A'],
      dtype='object', name='isbn', length=88392)

In [46]:
index_isbn_df = train.set_index(keys=['isbn'], inplace=False)
index_isbn_df

,user_id,rating
isbn,,
0002005018,8,4
0002005018,67544,7
0002005018,123629,8
0002005018,200273,8
0002005018,210926,9
...,...,...
0743525493,278843,7
067161746X,278851,6
0884159221,278851,7


In [50]:
rating_time = train['isbn'].value_counts()

In [55]:
rating_time.to_frame()

,isbn
0316666343,566
0971880107,465
0385504209,390
0312195516,307
0060928336,256
...,...
1854710443,1
1853915262,1
1853361208,1
1853263893,1


In [65]:
rating_time = rating_time.reset_index().rename(columns={'index':'isbn', 'isbn':'cnt'})

In [68]:
rating_time['cnt']=rating_time['cnt'].astype(str)
tmp = train.merge(rating_time, how="left", on="isbn")

In [72]:
tmp

,user_id,isbn,rating,cnt
0,8,0002005018,4,7
1,67544,0002005018,7,7
2,123629,0002005018,8,7
3,200273,0002005018,8,7
4,210926,0002005018,9,7
...,...,...,...,...
306790,278843,0743525493,7,1
306791,278851,067161746X,6,1
306792,278851,0884159221,7,1
306793,278851,0912333022,7,1


In [84]:
tmp.loc[tmp['cnt'] == '1', 'rating'] = tmp['rating']-(tmp['rating']-total_avg)*0.5831

In [85]:
tmp

,user_id,isbn,rating,cnt
0,8,0002005018,4.00000,7
1,67544,0002005018,7.00000,7
2,123629,0002005018,8.00000,7
3,200273,0002005018,8.00000,7
4,210926,0002005018,9.00000,7
...,...,...,...,...
306790,278843,0743525493,7.04065,1
306791,278851,067161746X,6.62375,1
306792,278851,0884159221,7.04065,1
306793,278851,0912333022,7.04065,1


In [87]:
train = tmp.drop(['cnt'], axis=1)

In [88]:
train

,user_id,isbn,rating
0,8,0002005018,4.00000
1,67544,0002005018,7.00000
2,123629,0002005018,8.00000
3,200273,0002005018,8.00000
4,210926,0002005018,9.00000
...,...,...,...
306790,278843,0743525493,7.04065
306791,278851,067161746X,6.62375
306792,278851,0884159221,7.04065
306793,278851,0912333022,7.04065
